## Bullet Bonds


A bullet bond is a type of fixed-income security with a very straightforward repayment structure.

### Key Features
1. **No Amortization**  
   - The issuer does not repay any portion of the principal during the life of the bond.  
   - Only interest (coupon) payments are made periodically (e.g., semi-annual or annual).

2. **Principal Repaid at Maturity**  
   - The entire face value (principal) is repaid in one "bullet" payment at the end of the bond’s term.  
   - This makes it different from amortizing bonds (like many mortgages), where principal is gradually repaid.

3. **Predictable Cash Flow**  
   - Investors receive regular coupon payments and then a large lump sum at maturity.  
   - This makes bullet bonds relatively simple to understand and model.

### Advantages
- **For issuers**: Lower cash outflow during the life of the bond since they don’t repay principal until maturity.  
- **For investors**: Predictable stream of income plus a known lump-sum repayment date.

### Risks
- **Credit risk**: Since principal repayment is concentrated at the end, investors bear the risk that the issuer might default before maturity.  
- **Refinancing risk for issuers**: They must have sufficient liquidity or refinancing ability to repay the lump sum when due.  

### Common Examples
- Many **corporate bonds** and **sovereign bonds** are structured as bullet bonds.  
- Eurobonds and U.S. Treasury notes/bonds are classic examples.



In [104]:
from pyfian.fixed_income.bond import BulletBond

bond = BulletBond('2020-01-01', '2025-01-01', 5, 4, notional=100)



from IPython.display import Markdown
from pandas import Timestamp

def display_bond_cash_flows(cash_flow_dict):
    table = "## Bond Cash Flows Table\n\n"
    table += "| Date       | Cash Flow Value |\n"
    table += "|------------|-----------------|\n"
    for time, value in sorted(cash_flow_dict.items(), key=lambda x: x[0]):
        table += f"| {time.strftime('%Y-%m-%d')} | {value:.2f}            |\n"
    display(Markdown(table))


display_bond_cash_flows(bond.payment_flow)



## Bond Cash Flows Table

| Date       | Cash Flow Value |
|------------|-----------------|
| 2020-04-01 | 1.25            |
| 2020-07-01 | 1.25            |
| 2020-10-01 | 1.25            |
| 2021-01-01 | 1.25            |
| 2021-04-01 | 1.25            |
| 2021-07-01 | 1.25            |
| 2021-10-01 | 1.25            |
| 2022-01-01 | 1.25            |
| 2022-04-01 | 1.25            |
| 2022-07-01 | 1.25            |
| 2022-10-01 | 1.25            |
| 2023-01-01 | 1.25            |
| 2023-04-01 | 1.25            |
| 2023-07-01 | 1.25            |
| 2023-10-01 | 1.25            |
| 2024-01-01 | 1.25            |
| 2024-04-01 | 1.25            |
| 2024-07-01 | 1.25            |
| 2024-10-01 | 1.25            |
| 2025-01-01 | 101.25            |


## Bond Day-Counting Conventions

These rules decide how to count days for calculating bond interest:

- **30/360**: Counts each month as 30 days, year as 360 days. Simple but less accurate. Used in some corporate bonds.
- **30E/360**: Like 30/360, but if a date is the 31st, it’s counted as the 30th. Used in Eurobonds.
- **Actual/Actual**: Counts real days in the period and year (365 or 366). Most accurate. Used in US Treasuries (ISDA) or international bonds (ICMA).
- **Actual/360**: Counts real days in the period, but uses a 360-day year. Common in money markets like loans.
- **Actual/365**: Counts real days in the period, uses a 365-day year. Used in some bond markets.
- **30/365**: Counts each month as 30 days, year as 365 days. Rare, used in specific financial contracts.
- **Actual/Actual-Bond**: Same as Actual/Actual (ICMA), counts real days in period and coupon period (e.g., half-year). Used in bond markets.

In [ ]:
bond_1 = BulletBond('2020-01-01', '2025-01-01', 5, 2, notional=100, day_count_convention = '30/360')
bond_2 = BulletBond('2020-01-01', '2025-01-01', 5, 2, notional=100, day_count_convention = '30e/360')
#bond_3 = BulletBond('2020-01-01', '2025-01-01', 5, 1, notional=100, day_count_convention = 'actual/actual')
bond_4 = BulletBond('2020-01-01', '2025-01-01', 5, 2, notional=100, day_count_convention = 'actual/360')
bond_5 = BulletBond('2020-01-01', '2025-01-01', 5, 2, notional=100, day_count_convention = 'actual/365')
bond_6 = BulletBond('2020-01-01', '2025-01-01', 5, 2, notional=100, day_count_convention = '30/365')
bond_7 = BulletBond('2020-01-01', '2025-01-01', 5, 2, notional=100, day_count_convention = 'actual/actual-Bond')

from IPython.display import Markdown

# --- Pretty day-count label helper ---
def dcc_label(dcc_obj):
    """
    Return a clean, human-readable label for common day-count objects.
    Works if the bond stores: a) a string like '30/360', or
    b) an instance like DayCount30360, DayCountActualActualBond, etc.
    """
    # If already a clean string, just use it
    if isinstance(dcc_obj, str):
        return dcc_obj

    # If it exposes a 'name' or similar
    for attr in ("name", "label", "code"):
        if hasattr(dcc_obj, attr):
            val = getattr(dcc_obj, attr)
            if isinstance(val, str) and val.strip():
                return val

    # Fall back to class name mapping
    cls = dcc_obj.__class__.__name__

    # Common mappings for pyfian-style names
    mapping = {
        "DayCount30360": "30/360",
        "DayCount30E360": "30E/360",
        "DayCountActual360": "ACT/360",
        "DayCountActual365": "ACT/365",
        "DayCount30365": "30/365",
        "DayCountActualActualBond": "ACT/ACT (Bond)",
        "DayCountActualActual": "ACT/ACT",
    }

    if cls in mapping:
        return mapping[cls]

    # Generic fallback: split CamelCase and add spaces
    import re
    spaced = re.sub(r"(?<!^)(?=[A-Z])", " ", cls)
    return spaced

# --- Display function in your style ---
def display_bond_accrued_interest(bond_dict, settlement_date="2023-03-01"):
    table = f"## Accrued Interest Table (Settlement Date: {settlement_date})\n\n"
    table += "| Bond   | Day Count Convention | Accrued Interest |\n"
    table += "|--------|----------------------|------------------|\n"

    for name, bond in bond_dict.items():
        # Try to fetch the day-count stored on the bond
        dcc = getattr(bond, "day_count_convention", None)
        label = dcc_label(dcc) if dcc is not None else "(unknown)"

        # Compute accrued interest (handle alternate param name)
        try:
            ai = bond.accrued_interest(settlement_date=settlement_date)
        except TypeError:
            ai = bond.accrued_interest(settlement=settlement_date)

        # Format numeric nicely
        ai_str = f"{ai:.6f}" if isinstance(ai, (int, float)) else str(ai)
        table += f"| {name} | {label} | {ai_str} |\n"

    display(Markdown(table))

# --- Usage with your bonds ---
bonds = {
    "bond_1": bond_1,  # 30/360
    "bond_2": bond_2,  # 30E/360
    "bond_4": bond_4,  # ACT/360
    "bond_5": bond_5,  # ACT/365
    "bond_6": bond_6,  # 30/365
    "bond_7": bond_7,  # ACT/ACT (Bond)
}

display_bond_accrued_interest(bonds, settlement_date="2023-03-01")




0.41666666666666663
0.41666666666666663
0.4097222222222222
0.4041095890410959
0.410958904109589
1.6298342541436464


## Accrued Interest Table (Settlement Date: 2023-03-01)

| Bond   | Day Count Convention | Accrued Interest |
|--------|----------------------|------------------|
| bond_1 | 30/360 | 0.416667 |
| bond_2 | 30E/360 | 0.416667 |
| bond_4 | actual/360 | 0.409722 |
| bond_5 | actual/365 | 0.404110 |
| bond_6 | 30/365 | 0.410959 |
| bond_7 | actual/actual-Bond | 1.629834 |


### Calculating Bond Price from Discount Curve

The price of a bond is the present value of its future cash flows, which consist of periodic coupon payments and the face value paid at maturity. A discount curve provides the discount factors needed to compute these present values, accounting for the time value of money and interest rate risk. Each cash flow is multiplied by the discount factor corresponding to its payment time, and the sum of these discounted cash flows gives the bond's price.

Consider a 5-year bond with:
- **Face Value**: $100
- **Annual Coupon Rate**: 5% (paid semi-annually, so $2.50 every 6 months)
- **Maturity**: 5 years

##### 1) Identify Cash Flows
- **Coupon Payments**: $2.50 at `t = 0.5, 1.0, ..., 4.5` years (semi-annual).
- **Final Payment**: $102.50 at `t = 5.0` years (face value + final coupon).

##### 2) Apply Discount Curve
Using the `DummyCurve` discount factor, defined as `discount_t(t) = 1 / (1 + 0.05 * t)`, each cash flow is discounted:
- At `t = 0.5`: $2.50 / (1 + 0.05 * 0.5) ≈ $2.439$
- At `t = 5.0`: $102.50 / (1 + 0.05 * 5.0) = $102.50 / 1.25 = $82.0$

##### 3) Calculate Bond Price
Sum the discounted cash flows:
- $2.439 + $2.381 + ... + $82.0 ≈ $102.066$

This total represents the bond's price, reflecting the present value of all future payments adjusted by the discount curve.

In [80]:
class DummyCurve:
             def discount_t(self, t):
                return 1 / (1 + 0.05 * t)

print(bond_1.value_with_curve(DummyCurve())[1])

from IPython.display import Markdown

def display_cash_flows_table(cash_flow_dict):
    """
    Display a Markdown table of discounted cash flows from a dictionary of time:value pairs.
    
    Args:
        cash_flow_dict (dict): Dictionary with time points (float) as keys and
                              discounted cash flow values (float) as values.
    """
    # Start the Markdown table
    table = "## Discounted Cash Flows Table\n\n"
    table += "| Time (Years) | Discounted Cash Flow Value |\n"
    table += "|--------------|----------------------------|\n"
    
    # Sort times to ensure chronological order
    for time in sorted(cash_flow_dict.keys()):
        value = cash_flow_dict[time]
        # Round to 4 decimal places, unless the value is a whole number (e.g., 82.0)
        formatted_value = f"{value:.4f}" if value < 10 else f"{value:.4f}".rstrip('0').rstrip('.')
        table += f"| {time:.1f}          | {formatted_value:>26} |\n"
    
    # Display the table
    display(Markdown(table))

display_cash_flows_table(bond_1.value_with_curve(DummyCurve())[1])


{0.5: 2.439024390243903, 1.0: 2.380952380952381, 1.5: 2.3255813953488373, 2.0: 2.2727272727272725, 2.5: 2.2222222222222223, 3.0: 2.173913043478261, 3.5: 2.127659574468085, 4.0: 2.0833333333333335, 4.5: 2.040816326530612, 5.0: 82.0}


## Discounted Cash Flows Table

| Time (Years) | Discounted Cash Flow Value |
|--------------|----------------------------|
| 0.5          |                     2.4390 |
| 1.0          |                     2.3810 |
| 1.5          |                     2.3256 |
| 2.0          |                     2.2727 |
| 2.5          |                     2.2222 |
| 3.0          |                     2.1739 |
| 3.5          |                     2.1277 |
| 4.0          |                     2.0833 |
| 4.5          |                     2.0408 |
| 5.0          |                         82 |
